In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from transformers import BertTokenizer
from scipy import stats
import numpy as np

plt.rcParams['figure.figsize'] = [14.4, 10.8]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower

RED_COLOR = '#ff6978'
BLUE_COLOR = '#2d93ad'

TOPS = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

FOLDER = './outputs/DS1/bert-base-multilingual-uncased'
CHECKPOINT = 'bert-base-multilingual-uncased'

tokenizer = BertTokenizer.from_pretrained(CHECKPOINT)
vocab_words = list(tokenizer.vocab.keys())

words_metrics = pd.read_csv(f'{FOLDER}/words_metrics.csv')
words_metrics

In [ ]:
lwo_df = pd.read_csv(f'{FOLDER}/lwo.csv')
lwo_df

In [ ]:
words_df = pd.read_csv(f'{FOLDER}/words/top_500_words.csv', sep=';')
words_df

In [ ]:
percentages = {}

for top in TOPS:
    percentages[f'lwo_{top}'] = {
        'average_attention': 0.0,
        'positive': 0.0,
        'negative': 0.0,
        'p-relevant': 0.0,
        'p-correct': 0.0,
    }
    
    top_absolute = words_df.head(top)
    words = top_absolute.word.values
    percentages[f'lwo_{top}']['average_attention'] = np.mean(top_absolute.absolute.values)

    # Just words that are in top-X words with absolute attention    
    subset = lwo_df[lwo_df.word.isin(words)]
    positive_words = subset[subset.f1 < 0]
    negative_words = subset[subset.f1 > 0]
    percentages[f'lwo_{top}']['positive'] = len(positive_words) / top * 100
    percentages[f'lwo_{top}']['negative'] = len(negative_words) / top * 100
    
    tfidf = pd.read_csv(f'{FOLDER}/words/relevant_{top}.csv')
    positive_relevant = positive_words[positive_words.word.isin(tfidf.word.values)]
    percentages[f'lwo_{top}']['p-relevant'] = len(positive_relevant) / top * 100
    
    correct = pd.read_csv(f'{FOLDER}/words/correct_{top}.csv')
    positive_correct = positive_words[positive_words.word.isin(correct.word.values)]
    if top == 250:
        positive_correct.to_csv(f'{FOLDER}/words/ploo_piw.csv', index=None)
    percentages[f'lwo_{top}']['p-correct'] = len(positive_correct) / top * 100
    
percentages_df = pd.DataFrame(percentages).T
percentages_df.insert(0, column='top', value=percentages_df.index)
percentages_df

In [ ]:
# Normality test:
for metric in ['average_attention', 'positive', 'p-relevant', 'p-correct']:
    print('%s: %.4f' % (metric, stats.shapiro(percentages_df[metric].values).pvalue))

In [ ]:
# Get correlations...
print(
    "Corr. attention vs positive: %.4f (p=%.4f)" %\
    stats.spearmanr(percentages_df.average_attention.values, percentages_df.positive.values)
)
print(
    "Corr. attention vs negative: %.4f (p=%.4f)" %\
    stats.spearmanr(percentages_df.average_attention.values, percentages_df.negative.values)
)
print(
    "Corr. attention vs p-relevant: %.4f (p=%.4f)" %\
    stats.spearmanr(percentages_df.average_attention.values, percentages_df['p-relevant'].values)
)
print(
    "Corr. attention vs p-correct: %.4f (p=%.4f)" %\
    stats.spearmanr(percentages_df.average_attention.values, percentages_df['p-correct'].values)
)

In [ ]:
# Top 30 positive/negative contributing words
TOP_N = 20

positive_words = lwo_df[lwo_df.f1 < 0].sort_values('f1', ascending=False)
negative_words = lwo_df[lwo_df.f1 > 0].sort_values('f1', ascending=False)

fig, axarr = plt.subplots(1, 2, figsize=(14.4, 8))
fig.subplots_adjust(wspace=0.35)

axarr[0].set_xlabel('Words that damages F1 when removed')
axarr[1].set_xlabel('Words that improve F1 when removed')

positive_words.head(TOP_N).plot(kind='barh', ax=axarr[0], x='word', y='f1', color=BLUE_COLOR, legend=None, xlabel='')
negative_words.head(TOP_N).plot(kind='barh', ax=axarr[1], x='word', y='f1', color=RED_COLOR, legend=None, xlabel='')